In [145]:
import numpy as np
from tqdm import tqdm

# Minesweeper Game Design For Solver Testing

In [103]:
class Minesweeper:
    
    DEFAULT = -98  # Empty tiles. (Haven't been chosen yet)
    BOMB = -100  # Bombs that are not yet labeled.
    LABELBOMB = -99  # Bombs the user has labeled.
    
    
    def __init__(self, length: int, width: int, bombs: int, custom_board=None):
        self.length = length
        self.width = width
        self.bombs = bombs

        # Create a new board if a custom board is not given.
        if custom_board is None:
            board = np.zeros((self.length, self.width)) + Minesweeper.DEFAULT
            
            shape = board.shape
            temp = board.flatten()
            temp[np.random.choice(temp.shape[0], self.bombs, replace=False)] = Minesweeper.BOMB
            temp = temp.reshape(shape)
            self.board = temp
            
        else:
            self.board = custom_board
        
        self.game_running = True
        self.history = []

    def valid(self, x: int, y: int):
        return 0 <= x < self.length and 0 <= y < self.width
        
    def adj(self, x: int, y: int):
        return self.board[max(0, x-1):min(self.length, x+2),max(0, y-1):min(self.width, y+2)]
        
    def reveal(self, x: int, y: int):
        def _reveal(xi, yi):
            if (not self.valid(xi, yi) 
                or self.board[xi, yi] >= 0 
                or self.board[xi, yi] in [Minesweeper.BOMB, Minesweeper.LABELBOMB]):
                return
            # Set the total to the sum of the bombs and the labeled bombs.
            self.board[xi, yi] = (np.sum(self.adj(xi, yi) == Minesweeper.BOMB) + 
                                  np.sum(self.adj(xi, yi) == Minesweeper.LABELBOMB))
            # Reveal the neighbors.
            if self.board[xi, yi] == 0:
                _reveal(xi - 1, yi - 1)
                _reveal(xi + 1, yi + 1)
                _reveal(xi + 1, yi - 1)
                _reveal(xi - 1, yi + 1)
                _reveal(xi, yi - 1)
                _reveal(xi, yi + 1)
                _reveal(xi - 1, yi)
                _reveal(xi + 1, yi)
    
        if not self.valid(x, y) or self.board[x, y] >= 0:
            return
        if self.board[x, y] == Minesweeper.BOMB or self.board[x, y] == Minesweeper.LABELBOMB:
            self.game_running = False
            return
        _reveal(x, y)
        
    def mark(self, x: int, y: int):
        if not self.valid(x, y):
            return True
        if self.board[x, y] != Minesweeper.BOMB:
            return False  # Wrong Move! A non bomb tile has been covered.
        self.board[x, y] = Minesweeper.LABELBOMB
        return True
    
    def user_view(self):
        board = np.copy(self.board)
        board[board == Minesweeper.BOMB] = Minesweeper.DEFAULT
        return board
    
    def revealed(self):
        return np.argwhere(self.board >= 0)
    
    def bombs_remaining(self):
        return np.sum(self.board == Minesweeper.BOMB)
    
    def game_won(self):
        return self.bombs_remaining() == 0
    
    def print(self):
        text = ""
        text += ("\n" + ("-" * 4) * self.length + "-") + "\n"
        for rows in self.board:
            text += "| "
            for col in rows:

                if int(col) == Minesweeper.DEFAULT:
                    char_to_print = f"{' ':^1}"
                elif int(col) == Minesweeper.BOMB:
                    char_to_print = f"{'•':^1}"
                elif int(col) == 0:
                    char_to_print = f"{'O':^1}"
                elif int(col) == Minesweeper.LABELBOMB:
                    char_to_print = f"{'⚐':^1}"
                else:
                    char_to_print = f"{int(col):^1}"

                text += f"{char_to_print} | "

            text += "\n" + ("-" * 4) * self.length + "-\n"
        print(text)
        
    def user_print(self):
        text = ""
        board = self.user_view()
        text += ("\n" + ("-" * 4) * self.length + "-") + "\n"
        for rows in board:
            text += "| "
            for col in rows:

                if int(col) == Minesweeper.DEFAULT:
                    char_to_print = f"{' ':^1}"
                elif int(col) == 0:
                    char_to_print = f"{'O':^1}"
                elif int(col) == Minesweeper.LABELBOMB:
                    char_to_print = f"{'⚐':^1}"
                else:
                    char_to_print = f"{int(col):^1}"

                text += f"{char_to_print} | "

            text += "\n" + ("-" * 4) * self.length + "-\n"
        print(text)
    
board = Minesweeper(5, 5, 5)
board.print()
board.reveal(0, 0)
board.print()
print(board.mark(2, 2))
board.print()


---------------------
|   |   | • |   |   | 
---------------------
|   |   |   |   | • | 
---------------------
| • |   |   |   |   | 
---------------------
| • |   |   |   |   | 
---------------------
| • |   |   |   |   | 
---------------------


---------------------
| O | 1 | • |   |   | 
---------------------
| 1 | 2 |   |   | • | 
---------------------
| • |   |   |   |   | 
---------------------
| • |   |   |   |   | 
---------------------
| • |   |   |   |   | 
---------------------

False

---------------------
| O | 1 | • |   |   | 
---------------------
| 1 | 2 |   |   | • | 
---------------------
| • |   |   |   |   | 
---------------------
| • |   |   |   |   | 
---------------------
| • |   |   |   |   | 
---------------------



## Identifying confirmed bombs to optimize probability matrix

In [104]:
def neighbors(board, x, y):
    l, w = board.shape
    return board[max(0, x-1):min(l, x+2),max(0, y-1):min(w, y+2)]

def find_bombs(board):
    l, w = board.shape
    labels = []
    for (x, y) in np.argwhere(board > 0):
        num_bombs = board[x, y] - np.sum(neighbors(board, x, y) == Minesweeper.LABELBOMB)
        num_squares = np.sum(neighbors(board, x, y) == Minesweeper.DEFAULT)
        if num_bombs == num_squares:
            indicies = [[x-1,y-1],[x-1,y],[x-1,y+1],[x,y-1],[x,y+1],[x+1,y-1],[x+1,y],[x+1,y+1]]
            for (xi, yi) in indicies:
                if 0 <= xi < l and 0 <= yi < w and board[xi, yi] == Minesweeper.DEFAULT:
                    labels.append((xi, yi))
    labels = set(labels)
    return labels

def mark_bombs():
    bomblist = find_bombs(board.user_view())
    for (x, y) in bomblist:
        if(not board.mark(x, y)):
            board.game_running = False
            print(x, y)
            break
    return bomblist

b = np.array([
    [2, Minesweeper.BOMB, Minesweeper.DEFAULT, Minesweeper.BOMB, 1],
    [2, Minesweeper.BOMB, Minesweeper.DEFAULT, Minesweeper.DEFAULT, 2],
    [1, 1, Minesweeper.DEFAULT, Minesweeper.BOMB, Minesweeper.DEFAULT],
    [1, 1, 2, Minesweeper.DEFAULT, Minesweeper.DEFAULT],
    [Minesweeper.BOMB, 1, 1, Minesweeper.BOMB, Minesweeper.DEFAULT],
])
board = Minesweeper(5, 5, np.sum((b == Minesweeper.BOMB)), custom_board=b)
board.print()
board.user_print()
print(board.game_running)
mark_bombs()
print(board.game_running)
board.user_print()


---------------------
| 2 | • |   | • | 1 | 
---------------------
| 2 | • |   |   | 2 | 
---------------------
| 1 | 1 |   | • |   | 
---------------------
| 1 | 1 | 2 |   |   | 
---------------------
| • | 1 | 1 | • |   | 
---------------------


---------------------
| 2 |   |   |   | 1 | 
---------------------
| 2 |   |   |   | 2 | 
---------------------
| 1 | 1 |   |   |   | 
---------------------
| 1 | 1 | 2 |   |   | 
---------------------
|   | 1 | 1 |   |   | 
---------------------

True
True

---------------------
| 2 | ⚐ |   |   | 1 | 
---------------------
| 2 | ⚐ |   |   | 2 | 
---------------------
| 1 | 1 |   |   |   | 
---------------------
| 1 | 1 | 2 |   |   | 
---------------------
| ⚐ | 1 | 1 |   |   | 
---------------------



## Calculate Probability Matrix

In [130]:
board = Minesweeper(5, 5, 5)
user_board = board.user_view()

def calculate_matrix(board, revealed_indices):
    matrix = np.zeros_like(board) + 0.0
    l, w = board.shape
    
    for (x, y) in revealed_indices:
        num_bombs = board[x, y] - np.sum(neighbors(board, x, y) == Minesweeper.LABELBOMB)
        num_squares = np.sum(neighbors(board, x, y) == Minesweeper.DEFAULT)

        
        if num_squares == 0:
            prob = 0.0  # There aren't any squares for the revealed index, don't do anything.
        elif num_bombs / num_squares == 0:
            prob = -10  # Choose these squares, they r the safest options.
        else:
            prob = num_bombs / num_squares

        matrix[max(0, x-1):min(l, x+2),max(0, y-1):min(w, y+2)] += prob
        
    indices = np.argwhere(board >= 0)
    matrix[indices[:, 0], indices[:, 1]] = 0

    indices = np.argwhere(board == Minesweeper.LABELBOMB)
    matrix[indices[:, 0], indices[:, 1]] = 0

    matrix[matrix == 0.0] = 10
    return matrix

guess, matrix = make_guess()
print(guess)
# board.user_print()
print(board.game_running)
print(matrix)

[0 0]
True
[[10. 10. 10. 10. 10.]
 [10. 10. 10. 10. 10.]
 [10. 10. 10. 10. 10.]
 [10. 10. 10. 10. 10.]
 [10. 10. 10. 10. 10.]]


In [106]:
mark_bombs()
board.print()


---------------------
| • | 2 | • | • |   | 
---------------------
|   |   |   |   |   | 
---------------------
|   |   |   | • |   | 
---------------------
|   |   |   |   |   | 
---------------------
|   |   |   |   | • | 
---------------------



## Putting the 2 steps together

In [107]:
def make_guess():
    mark_bombs()
    matrix = calculate_matrix(board.user_view(), board.revealed())
    lowest = np.argwhere(matrix == np.min(matrix))
    np.random.shuffle(lowest)
    guess = lowest[0]
    board.reveal(guess[0], guess[1])
    return guess, matrix

In [142]:
board = Minesweeper(16, 30, 99)
# board.print()

num_moves = 0
while board.game_running and not board.game_won():
    guess, matrix = make_guess()
    print(guess)
#     board.print()
    num_moves += 1

if board.game_won():
    print("Game Won!")
else:
    print(f"Game Lost! {num_moves}")

[ 3 13]
Game Lost! 1


In [149]:
wins = 0
draws = 0
games = 100
failed_due_to_guessing_wrong = 0

for _ in tqdm(range(games), ncols=80):
    board = Minesweeper(16, 30, 99)
    num_moves = 0
    num_guesses = 0
    while board.game_running and not board.game_won():
        guess, matrix = make_guess()
        num_guesses = np.argwhere(matrix == np.min(matrix)).shape[0]
        num_moves += 1
    
    if board.game_won():
        wins += 1
    else:
        if num_moves <= 10:
            draws += 1
        else:
            if num_guesses > 1:
                failed_due_to_guessing_wrong += 1
            
            
print(f"Wins: {wins}\tDraws: {draws}\tNum Guesses: {failed_due_to_guessing_wrong}")

100%|█████████████████████████████████████████| 100/100 [00:09<00:00, 10.27it/s]

Wins: 1	Draws: 65	Num Guesses: 29
